In [34]:
import datetime
import joblib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.pyplot import figure
from matplotlib.ticker import NullFormatter  # useful for `logit` scale
import numpy as np
import pandas as pd
import PySimpleGUI as sg
import seaborn as sns
from sklearn.ensemble import ExtraTreesRegressor
from windrose import WindroseAxes
import WindScrptB7 
import WinFunctB7 as WFB7
import time

In [35]:
matplotlib.use('TkAgg')
WSB7=WindScrptB7.PlotScript()

In [36]:
def ReadDatasets(Dataset,gen):
    load_model = joblib.load(gen)
    turbine=pd.read_csv(Dataset)
    if turbine.isnull().sum().sum()==0:
        sg.SystemTray.notify('Wind Power Analysis', 'Dataset Ok')
    else:
        sg.SystemTray.notify('Wind Power Analysis', "Atention - Null Values!!!!!")
    dfTurbine=turbine.copy()
    dfTurbine['Date/Time']=pd.to_datetime(dfTurbine['Date/Time'], format='%d %m %Y %H:%M')
    dfTurbine.rename(columns={'Date/Time':'Date',
                              'LV ActivePower (kW)':'ActPower',
                              'Wind Speed (m/s)':'WindSpd',
                              'Theoretical_Power_Curve (KWh)':'ThePower',
                              'Wind Direction (°)':'WindDir'},inplace=True)
    dfTurbine['Day']= dfTurbine['Date'].dt.day
    dfTurbine['Month']= dfTurbine['Date'].dt.month
    dfTurbine['Year']= dfTurbine['Date'].dt.year
    dfTurbine['Hour']= dfTurbine['Date'].dt.hour
    dfTurbine['Minute']= dfTurbine['Date'].dt.minute
    dfTurbine.sort_values(['Year','Month','Day','Hour','Minute'], ascending=True)
    dfTurbine['CumRealPower'] = dfTurbine['ActPower'].cumsum()
    dfTurbine['CumTheoPower'] = dfTurbine['ThePower'].cumsum()
    return turbine,load_model,dfTurbine

In [37]:
def LabelStat(dfTurbine,GenParam):
    WFB7.labelStatusGen(dfTurbine,PowUpperLever=GenParam['LimUpper'],PowLowerLever=GenParam['LimLower'],CutIn=GenParam['CutIn'],CutinPower=load_model.predict([[GenParam['CutIn']]]))
    stat=WFB7.StatVal(dfTurbine)        #statistic data
    for j in range(1):
        daysdic={'Wind':[stat['Inside'][j]+stat['Outside'][j]+stat['OutOfService'][j]],
                 'WhithoutWin':[stat['WhithoutWin'][j]]}
        statWin={'Inside_Range':[stat['Inside'][j]],
                 'Outside_Range':[stat['Outside'][j]],
                 'Out_Of_Service':[stat['OutOfService'][j]]}
    return daysdic,statWin

In [38]:
def delete_fig_agg(fig_agg):
    fig_agg.get_tk_widget().forget()
    plt.close('all')


In [39]:
fig_canvas_agg=0

markers={'WithoutWind':'gray','Inside':'green','Outside':'yellow','OutOfService':'red'}
units_measure={'ActPower':'KW',
               'WindSpd':'m/s',
               'ThePower':'kWh',
               'WinDir':'º'}
               
GenParam={'CutIn':3.5,
          'Sat': 12,
          'CutOf':25,
          'LimLower':0.95,
          'LimUpper':1.05}   

script1={'Menu': [['&File', ['&Open','&Save','&Exit',]],
                  ['&Edit', ['&Cut','&Copy','&Paste',]],
                  ['&Parameters',['&Dataset','&Generator',]],
                  ['&Help',['&Help','&About Us']]],
         'TilW':  'Wind Power Analysis <-> https://github.com/Bruno-H-Neves',
         'STlW':  'Prodution Graph',
         'Bts1': ('Energy','Time','GenCv','Wind','W_Mon','G_Mon','WinOnOff','WindOnGenOff'),
         'Bts':  ('Go','Exit','Ok'),
         'loc':  (5,5),         
         'sz':  (1260,640)}

In [40]:
TabYear = [[sg.T('2018')]]
TabMonth = [[sg.T('Month')]]
TabRange = [[sg.T('In Construction')]]
TabWind = [[sg.T(key='-WindSt-')]]
TabWindOn = [[sg.T(key='-WindOn-')]]

In [41]:
layoutMain= [[sg.Text(key='-Datafile-',size=(50,1),font='Helvetica 8')],
             [sg.Text(key='-Genfile-',size=(50,1),font='Helvetica 8')],
             [sg.Frame('     CutIn       Sat     CutOut       MinL     MaxL',[[sg.Slider(range=(0,30), resolution=.1, default_value=GenParam['CutIn'], 
                                                          orientation='v', key='-CutIn-', size=(10,15)),
                                                sg.Slider(range=(0,30), resolution=.1, default_value=GenParam['Sat'], 
                                                          orientation='v', key='-Sat-', size=(10,15)),
                                                sg.Slider(range=(0,30), resolution=.1, default_value=GenParam['CutOf'],
                                                          orientation='v', key='-CutOu-', size=(10,15)),
                                                sg.Slider(range=(0.5,1), resolution=.01, default_value=GenParam['LimLower'],
                                                          orientation='v', key='-LLow-', size=(10,15)),
                                                sg.Slider(range=(1,1.3), resolution=.01, default_value=GenParam['LimUpper'],
                                                          orientation='v', key='-Lupp-', size=(10,15))]],size=(275,200),title_color='black'),
                                                sg.Button(script1['Bts'][0],button_color=('white', 'blue'))],
             [sg.Frame('Date Selector',[[sg.TabGroup([[sg.Tab('Year',TabYear), 
                                                       sg.Tab('Month', TabMonth),
                                                       sg.Tab('Range', TabRange)]])]],size=(325,100),title_color='black')],
             [sg.Frame('Results (Hours)',[[sg.TabGroup([[sg.Tab('Wind Status', TabWind), 
                                                         sg.Tab('Wind On', TabWindOn)]])]],size=(325,250),title_color='black')]]


In [42]:
layout = [[sg.Radio(script1['Bts1'][0],"RADIO1", key=script1['Bts1'][0], size=(6,1)),
           sg.Radio(script1['Bts1'][1],"RADIO1", key=script1['Bts1'][1], size=(6,1)),
           sg.Radio(script1['Bts1'][2],"RADIO1", key=script1['Bts1'][2], default=True, size=(6,1)),
           sg.Radio(script1['Bts1'][3],"RADIO1", key=script1['Bts1'][3], size=(6,1)),
           sg.Radio(script1['Bts1'][4],"RADIO1", key=script1['Bts1'][4], size=(6,1)),
           sg.Radio(script1['Bts1'][5],"RADIO1", key=script1['Bts1'][5], size=(6,1)),
           sg.Radio(script1['Bts1'][6],"RADIO1", key=script1['Bts1'][6], size=(6,1)),
           sg.Radio(script1['Bts1'][7],"RADIO1", key=script1['Bts1'][7], size=(6,1)),
           sg.Radio('XXX',"RADIO1", key='XXX', size=(6,1)),sg.Button('Show', size=(10,1),button_color=('white', 'blue'))],
         [sg.Text(key='-TitleGraph-',size=(70,1),font='Helvetica 14',text_color='black',justification='center')],
         [sg.Canvas(key='-CANVAS-')]]

In [43]:
GenLayout=[[sg.Menu(script1['Menu'], tearoff=True)],
           [sg.Frame(layout=layoutMain,title='Define Parameters',title_color='black', 
                     tooltip="left",size=(325,600)),
            sg.Frame(layout=layout,title='Graph',title_color='black',
                     size=(900,600), tooltip="left")]]

In [44]:
sg.theme('DarkTeal11')
Dataset = sg.popup_get_file('File Name','Select Dataset') 
gen = sg.popup_get_file('File Name','Select Turbine Parameters') 
if Dataset==None or gen==None:
    sg.SystemTray.notify('Wind Power Analysis', 'Empty Dataset')
else:
    _,load_model,dfTurbine=ReadDatasets(Dataset,gen)
    daysdic,statWin=LabelStat(dfTurbine,GenParam)
window = sg.Window(script1['TilW'], GenLayout, finalize=True, element_justification='left', 
                   font='Helvetica 10',no_titlebar=False, location=script1['loc'],size=script1['sz'])
sg.SystemTray.notify('Wind Power Analysis', 'Running Trial Version')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator ExtraTreeRegressor from version 1.0.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator ExtraTreeRegressor from version 1.0.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator ExtraTreeRegressor from version 1.0.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator ExtraTreeRegressor from version 1.0.2 when using version 0.24.1. This might lead to breaking code or inva

'__TIMEOUT__'

In [45]:
while True:
    event, values = window.read()
    time.sleep(1)

    if event in ('Exit', None):
        break
    if event is None:
        break
    if event== 'Dataset':
        Dataset = sg.popup_get_file('File Name','Select Dataset') 
    if event== 'Generator':
        gen = sg.popup_get_file('File Name','Select Turbine Parameter') 
    if event== 'Dataset' or  event== 'Generator':
        if Dataset==None or gen==None:
            sg.SystemTray.notify('Wind Power Analysis', 'Empty Dataset')
        else:
            _,load_model,dfTurbine=ReadDatasets(Dataset,gen)
            daysdic,statWin=LabelStat(dfTurbine,GenParam)
    if Dataset!=None and gen!=None:
        GenParam['CutIn']=values['-CutIn-']
        GenParam['CutOf']=values['-CutOu-']
        GenParam['Sat']=values['-Sat-']
        GenParam['LimUpper']=values['-Lupp-']
        GenParam['LimLower']=values['-LLow-']
        daysdic,statWin=LabelStat(dfTurbine,GenParam)
        if event in ('Go',None) or event in ('Show',None) :
            if fig_canvas_agg !=0:
                delete_fig_agg(fig_canvas_agg)
            time.sleep(1)
            window['-Datafile-'].update(Dataset)
            window['-Genfile-'].update(gen)
            window['-WindSt-'].update(f"Wind:{daysdic['Wind'][0]} -> {round(100*daysdic['Wind'][0]/(daysdic['Wind'][0]+daysdic['WhithoutWin'][0]))} % \nWind Off:{daysdic['WhithoutWin'][0]} -> {round(100*daysdic['WhithoutWin'][0]/(daysdic['Wind'][0]+daysdic['WhithoutWin'][0]))} %")
            window['-WindOn-'].update(f"Inside:{statWin['Inside_Range'][0]} -> {round(100*statWin['Inside_Range'][0]/(statWin['Inside_Range'][0]+statWin['Outside_Range'][0]+statWin['Out_Of_Service'][0]))} % \nOutside:{statWin['Outside_Range'][0]} -> {round(100*statWin['Outside_Range'][0]/(statWin['Inside_Range'][0]+statWin['Outside_Range'][0]+statWin['Out_Of_Service'][0]))} %\nOut of Service:{statWin['Out_Of_Service'][0]} -> {round(100*statWin['Out_Of_Service'][0]/(statWin['Inside_Range'][0]+statWin['Outside_Range'][0]+statWin['Out_Of_Service'][0]))} %")
            daysdic,statWin=LabelStat(dfTurbine,GenParam)
            WFB7.labelStatusGen(dfTurbine,PowUpperLever=GenParam['LimUpper'],PowLowerLever=GenParam['LimLower'],CutIn=GenParam['CutIn'],CutinPower=load_model.predict([[GenParam['CutIn']]]))
            stat=WFB7.StatVal(dfTurbine)
            if values[script1['Bts1'][0]]==True:
                window['-TitleGraph-'].update("Wind Turbine Power Production: Theoretical vs Real vs Not Produced")
                fig=WFB7.plotdata(dfTurbine,Y1='CumRealPower',Y2='CumTheoPower',Y3='ActPower',Y4='ThePower',show=0,script=1)
            elif values[script1['Bts1'][1]]==True:
                window['-TitleGraph-'].update("Wind Turbine: Power Production, Wind Speed")
                fig=WFB7.plotdata1(dfTurbine,show=0,script=1)
            elif values[script1['Bts1'][2]]==True:
                window['-TitleGraph-'].update("Wind Turbine: Production Diagram")
                fig=WFB7.pltpower(dfTurbine,show=0,CutIn=GenParam['CutIn'],Saturation=GenParam['Sat'],CutOff=GenParam['CutOf'],markers=markers,script=1)
            elif values[script1['Bts1'][3]]==True:
                window['-TitleGraph-'].update("Wind Direction and Intensity")
                fig=WFB7.plotWindrose(dfTurbine,show=0,script=1)
            elif values[script1['Bts1'][4]]==True:
                window['-TitleGraph-'].update("Wind Direction and Intensit-Month")
                fig=WFB7.plotPowerWin(dfTurbine,show=0,script=1)
            elif values[script1['Bts1'][5]]==True:
                window['-TitleGraph-'].update("Wind Analysis (spd dir) vs turbine operation mode-Month")               
                fig=WFB7.PlotMonth(dfTurbine,markers,CutIn=GenParam['CutIn'],Saturation=GenParam['Sat'],CutOff=GenParam['CutOf'],show=0,script=1)
            elif values[script1['Bts1'][6]]==True:
                window['-TitleGraph-'].update("Wind Turbine: Production Wind Diagram")               
                fig=WFB7.PlotGen(dfTurbine,markers,CutIn=GenParam['CutIn'],Saturation=GenParam['Sat'],CutOff=GenParam['CutOf'],show=0,script=1)
            elif values[script1['Bts1'][7]]==True:
                window['-TitleGraph-'].update("Wind Turbine: Production Wind Diagram-2")               
                fig=WFB7.plot_dual(dfTurbine,markers,CutIn=GenParam['CutIn'],Saturation=GenParam['Sat'],CutOff=GenParam['CutOf'],month=1,show=0,script=1)
            fig_canvas_agg = WSB7.draw_figure(window['-CANVAS-'].TKCanvas,fig)
sg.SystemTray.notify('Wind Power Analysis', 'Trial Version Finished')
window.close()
